In [44]:
import pandas as pd
import numpy as np
import math
from Simplex import Simplex


In [45]:
#Load Data
_1Q_2020 = pd.read_csv('../data/metro-bike-share-trips-2020-q1.csv')
_3Q_2020 = pd.read_csv('../data/metro-trips-2020-q3.csv')
_4Q_2020 = pd.read_csv('../data/metro-trips-2020-q4.csv')
_1Q_2019 = pd.read_csv('../data/metro-bike-share-trips-2019-q4.csv')

In [46]:
def label_charged(duration):
    """"Function to create a new catergorical variable that indicate if theres has to be an adittional charge"""
    if duration <= 30:
        return 0
    else:
        return 1
    
def cambiar_minusculas_variable(archivo,variable):
    archivo[variable]=archivo[variable].str.lower()

In [47]:
#Concatenate the 4 datasets
data = pd.concat([_1Q_2020,_3Q_2020,_4Q_2020,_1Q_2019])

In [48]:
# Change to lower categorical variables
cambiar_minusculas_variable(data,'trip_route_category')
cambiar_minusculas_variable(data,'passholder_type')
cambiar_minusculas_variable(data,'bike_type')


In [49]:
data['start_time'] = pd.to_datetime(data['start_time'])
data['end_time'] = pd.to_datetime(data['end_time'])

In [50]:
data = data.drop(data[data['passholder_type']=="testing"].index)

In [51]:
data = data[data['passholder_type'].notna()]

In [52]:
data['charged_ride'] = data['duration'].apply(label_charged)

In [53]:
# Column for hour of day of ride.
data['start_hour'] = data['start_time'].dt.hour


In [54]:
bike_last_used_id = bike_last_used = data.groupby('bike_id')['end_time'].max().keys().tolist()
bike_last_used_date = data.groupby('bike_id')['end_time'].max().tolist()

In [55]:
latest_ride_date = data['end_time'].max()
# Create feature indicating active bikes, used in past 6 months. 1 if active, 0 if inactive.
inactive_ids = [record[0] for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
                record[1] < latest_ride_date + np.timedelta64(-6, 'M') ]
data['active_6m'] = data['bike_id'].apply(lambda bike_id: 0 if bike_id in (inactive_ids) else 1)

In [56]:
#Select variables to modeling
data = data [["trip_id", "duration","start_time", "end_time", "trip_route_category", "passholder_type"]]

In [57]:

data = data.reset_index()

In [58]:
data

,index,trip_id,duration,start_time,end_time,trip_route_category,passholder_type
0,0,134867493,25,2020-01-01 00:16:00,2020-01-01 00:41:00,one way,monthly pass
1,1,134867799,35,2020-01-01 00:24:00,2020-01-01 00:59:00,one way,one day pass
2,2,134868104,37,2020-01-01 00:31:00,2020-01-01 01:08:00,one way,walk-up
3,3,134868103,36,2020-01-01 00:32:00,2020-01-01 01:08:00,one way,walk-up
4,4,134868102,35,2020-01-01 00:33:00,2020-01-01 01:08:00,one way,walk-up
...,...,...,...,...,...,...,...
227232,76310,134982544,1440,2019-12-31 23:35:51,2020-01-02 17:13:50,round trip,monthly pass
227233,76311,134866394,9,2019-12-31 23:41:52,2019-12-31 23:50:58,one way,monthly pass
227234,76312,134866292,4,2019-12-31 23:43:19,2019-12-31 23:47:41,one way,annual pass
227235,76313,134866392,5,2019-12-31 23:48:17,2019-12-31 23:53:55,one way,annual pass


In [59]:
# Create dummy variables for passholder type.
one_hot_pass = pd.get_dummies(data['passholder_type']).rename(columns={
    'annual pass' : 'annual',
    'monthly pass' : 'monthly',
    'one day pass' : 'one_day',
    'walk-up' : 'walk_up'
})

# Create dummy variables for trip route category.
one_hot_trip_type = pd.get_dummies(data['trip_route_category']).rename(columns = {
    'one way' : 'one_way',
    'round trip' : 'round_trip'
})

data = data.join([one_hot_pass, one_hot_trip_type])

In [60]:
# Creates feature for 30 minute blocks that walk-up users have to pay for.
# Creates another feature for 30 minute blocks for passholders that exclude first 30 minute block.
data['time_block_count'] = data['duration'].apply(lambda duration: math.ceil(duration/30))
data['time_block_count_post_free'] = data['duration'].apply(lambda duration: \
                                                                        math.ceil(duration/30)) - 1

In [61]:
pass_groups = data.groupby('passholder_type')
pass_groups.groups.keys()

agg_pass_groups = pass_groups.agg({
    'trip_id' : 'count',
    'walk_up' : 'sum',
    'time_block_count' : 'sum',
    'time_block_count_post_free' : 'sum',
    'duration' : 'sum'
}).rename(columns = {
    'trip_id' : 'total_trips',
    'walk_up' : 'pay_count_walk_up',
    'time_block_count' : 'total_30min_blocks',
    'time_block_count_post_free' : 'total_30min_blocks_post_free',
    'duration' : 'total_minutes'
}).reset_index()

agg_pass_groups['passholder_type'] = ['annual','monthly','daily','walk_up']
agg_pass_groups.index = agg_pass_groups['passholder_type']
agg_pass_groups = agg_pass_groups.drop('passholder_type',1)

In [62]:
# Passes sold = Total rides / Rides to break even
price_wo_pass = 3.5
price_w_pass_day = 0
price_w_pass_month = 0
price_w_pass_annual = 1.75

# Average difference between passholders and walk-up riders across previous and current pricing.
# This is represented by the difference in prices per 30 minute block since pass holders get the first 30 minutes free.
# As current data is from before the price change, previous prices are used to estimate number of passes.

day_diff = price_wo_pass - price_w_pass_day
month_diff = price_wo_pass - price_w_pass_month
annual_diff = price_wo_pass - price_w_pass_annual

# Prices of passes.

day_pass_price = 7
month_pass_price = 20
annual_pass_price = 40

# Estimated rides required for breakeven.

rides_day_pass_estimate = day_pass_price / day_diff
rides_month_pass_estimate = month_pass_price / month_diff
rides_annual_pass_estimate = annual_pass_price / annual_diff


In [63]:
#Para estimaciones conservadoras, escalamos las tres estimaciones por un factor común del 40%.
rides_day_pass_estimate *= 1.4
rides_month_pass_estimate *= 1.4
rides_annual_pass_estimate *= 1.4

In [64]:
# of passholders.

passes_sold_day = math.ceil(agg_pass_groups.loc['daily','total_trips']/rides_day_pass_estimate)
passes_sold_month = math.ceil(agg_pass_groups.loc['monthly','total_trips']/rides_month_pass_estimate)
passes_sold_annual = math.ceil(agg_pass_groups.loc['annual','total_trips']/rides_annual_pass_estimate)
passes_sold_total = passes_sold_day + passes_sold_month + passes_sold_annual

agg_pass_groups['passes_sold'] = [passes_sold_annual, passes_sold_month, passes_sold_day, 0]


In [65]:
agg_pass_groups['previous_pass_prices'] = [40, 20, 7, 0]
agg_pass_groups['new_pass_prices'] = [40,17, 5, 0]
agg_pass_groups['average_charged_blocks_per_ride'] = agg_pass_groups['total_30min_blocks_post_free'] / agg_pass_groups['total_trips']
agg_pass_groups['percent_total_trips'] = agg_pass_groups['total_trips']/sum(agg_pass_groups['total_trips'])

In [66]:
current_tot_rev = (sum(3.5 * agg_pass_groups['pay_count_walk_up']) 
                   + sum(1.75 * agg_pass_groups['total_30min_blocks_post_free'])
                   + 1.75 * (agg_pass_groups['total_30min_blocks'] ['annual']
                                            - agg_pass_groups['total_30min_blocks_post_free']['annual'])
                   + sum(agg_pass_groups['passes_sold'] * agg_pass_groups['previous_pass_prices']))

In [67]:

#Se declaran los precios
previous_walk_up = 3.5
previous_pass = 1.75
# Revenue for walk ups


In [68]:
lhs_eq = [[0, 1, 1, 0, 0, 0, 0],
          [0, 0, 0, 1, 1, 0, 0],
          [0, 0, 0, 0, 0, 1, 1]]
rhs_eq = [1, 1, 1]

walk_up_rev_block = previous_walk_up * agg_pass_groups.loc['walk_up','total_30min_blocks']

# Revenue for pass sales.
trip_rev_day_pass = agg_pass_groups.loc['daily', 'total_30min_blocks_post_free'] * previous_pass
trip_rev_month_pass = agg_pass_groups.loc['monthly', 'total_30min_blocks_post_free'] * previous_pass
trip_rev_annual_pass = agg_pass_groups.loc['annual', 'total_30min_blocks'] * previous_pass

# Revenue for pass sales is pass is cancelled.
trip_rev_day_no_pass = agg_pass_groups.loc['daily', 'total_30min_blocks'] * previous_walk_up * 0.7
trip_rev_month_no_pass = agg_pass_groups.loc['monthly', 'total_30min_blocks'] * previous_walk_up * 0.7
trip_rev_annual_no_pass = agg_pass_groups.loc['annual', 'total_30min_blocks'] * previous_walk_up * 0.7

# Revenue for sales of passes.
pass_sales_rev_day = (agg_pass_groups.loc['daily','previous_pass_prices'] 
                      * agg_pass_groups.loc['daily','passes_sold'])
pass_sales_rev_month = (agg_pass_groups.loc['monthly','previous_pass_prices'] 
                        * agg_pass_groups.loc['monthly','passes_sold'])
pass_sales_rev_annual = (agg_pass_groups.loc['annual','previous_pass_prices'] 
                         * agg_pass_groups.loc['annual','passes_sold'])

obj_1 = [-walk_up_rev_block, -(trip_rev_day_pass + pass_sales_rev_day), trip_rev_day_no_pass, -(trip_rev_month_pass + pass_sales_rev_month), -trip_rev_month_no_pass, -(trip_rev_annual_pass + pass_sales_rev_annual), -trip_rev_annual_no_pass]

In [69]:
c = np.array(obj_1)
A =  np.array(lhs_eq)
b =  np.array(rhs_eq)

In [70]:
print(A)
print(b)
print(c)

[[0 1 1 0 0 0 0]
 [0 0 0 1 1 0 0]
 [0 0 0 0 0 1 1]]
[1 1 1]
[-393620.5  -125952.75 -157378.2  -422192.75 -441465.5   -79897.75
  -69856.85]


In [73]:
model = Simplex(c,A,b,"Max")

In [74]:
model.solve()

[0, 0, 0, 0, 0, 0, 0]